In [23]:
import mne
import fooof
import numpy as np
import os.path as op
import pickle
import saflow
import saflow
import os.path as op
import pickle as pkl
import numpy as np
import pandas as pd
import os
import mne_bids
import mne
from sklearn.model_selection import permutation_test_score, LeaveOneGroupOut
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from scipy import stats
import matplotlib.pyplot as plt
import fooof
from statsmodels.stats.multitest import fdrcorrection
import random
from matplotlib.backends.backend_pdf import PdfPages

In [24]:
output_fname = op.join('/home/hyruuk/GitHub/cocolab/cc_saflow/tmp/', 'subject_averaged_fooofs.pkl')
with open(output_fname, 'rb') as f:
    fooof_array = pickle.load(f)

In [25]:
def average_bands_from_bands(fm, freqs=saflow.FREQS):
    fooof_freqs = fm.freqs
    original_bands = []
    fooof_bands = []
    original_corrected = []
    for band in freqs:
        freq_mask = np.where((fooof_freqs >= band[0]) & (fooof_freqs <= band[1]))[0]
        original = np.mean(fm.power_spectrum[freq_mask])
        original_bands.append(original)

        fooof = np.mean(fm.fooofed_spectrum_[freq_mask])
        fooof_bands.append(fooof)

        corrected = fm.power_spectrum - fm._ap_fit
        original_corrected.append(np.mean(corrected[freq_mask]))
    
    exponent = fm.get_params('aperiodic_params', 'exponent')
    offset = fm.get_params('aperiodic_params', 'offset')
    r_squared = fm.get_params('r_squared')
    return original_bands, fooof_bands, original_corrected, exponent, offset, r_squared

def get_data_matrix(fooof_array):
    original_bands_array = []
    fooof_bands_array = []
    original_corrected_array = []
    exponent_array = []
    offset_array = []
    r_squared_array = []

    for cond_idx in range(fooof_array.shape[0]):
        original_bands_subj = []
        fooof_bands_subj = []
        original_corrected_subj = []
        exponent_subj = []
        offset_subj = []
        r_squared_subj = []
        for subj_idx in range(fooof_array.shape[1]):
            original_bands_chans = []
            fooof_bands_chans = []
            original_corrected_chans = []
            exponent_chans = []
            offset_chans = []
            r_squared_chans = []
            for chan_idx in range(fooof_array.shape[2]):
                fm = fooof_array[cond_idx, subj_idx, chan_idx]
                original_bands, fooof_bands, original_corrected, exponent, offset, r_squared = average_bands_from_bands(fm)
                original_bands_chans.append(original_bands)
                fooof_bands_chans.append(fooof_bands)
                original_corrected_chans.append(original_corrected)
                exponent_chans.append(exponent)
                offset_chans.append(offset)
                r_squared_chans.append(r_squared)
            original_bands_subj.append(original_bands_chans)
            fooof_bands_subj.append(fooof_bands_chans)
            original_corrected_subj.append(original_corrected_chans)
            exponent_subj.append(exponent_chans)
            offset_subj.append(offset_chans)
            r_squared_subj.append(r_squared_chans)
        original_bands_array.append(original_bands_subj)
        fooof_bands_array.append(fooof_bands_subj)
        original_corrected_array.append(original_corrected_subj)
        exponent_array.append(exponent_subj)
        offset_array.append(offset_subj)
        r_squared_array.append(r_squared_subj)

        data = {'original_bands': np.array(original_bands_array),
                'fooof_bands': np.array(fooof_bands_array),
                'original_corrected': np.array(original_corrected_array),
                'exponent': np.array(exponent_array),
                'offset': np.array(offset_array),
                'r_squared': np.array(r_squared_array)}
    return data

data = get_data_matrix(fooof_array)

In [26]:
data['original_bands'].shape

(2, 32, 270, 8)

In [27]:
def format_data_for_classif(data, feature='original_bands'):
    X = []
    y = []
    groups = []
    for cond_idx in range(data[feature].shape[0]):
        for subj_idx in range(data[feature].shape[1]):
            X.append(data[feature][cond_idx, subj_idx])
            y.append(cond_idx)
            groups.append(subj_idx)
    X = np.array(X).transpose(2, 0, 1)
    y = np.array(y)
    groups = np.array(groups)
    return X, y, groups

X, y, groups = format_data_for_classif(data, feature='original_bands')

In [30]:
X_contrast, tvals, pvals = simple_contrast(X, y, groups)


NameError: name 'y_avg' is not defined

In [29]:
def simple_contrast(X, y, groups):
    """Computes subject-averages and contrasts between conditions."""
    n_features = X.shape[0]
    # Average each condition separately
    X_avg_by_cond = []
    for cond in np.unique(y):
        X_avg_by_cond.append(np.mean(X[y == cond], axis=0))
    # Compute normalized contrast (A - B)/B
    X_contrast = (X_avg_by_cond[0] - X_avg_by_cond[1]) / X_avg_by_cond[1]
    # Split conditions for ttest
    X_condA = X[y == 0]
    X_condB = X[y == 1]
    
    # Compute t-test
    tvals = []
    pvals = []
    for feature_idx in range(n_features):
        t, p = stats.ttest_rel(X_condA[:,feature_idx,:], X_condB[:,feature_idx,:], axis=0)
        tvals.append(t)
        pvals.append(p)
    tvals = np.array(tvals)
    pvals = np.array(pvals)
    return X_contrast, tvals, pvals


def singlefeat_classif(clf, cv, X, y, groups, n_perms=1):
    clf = LinearDiscriminantAnalysis()
    cv = LeaveOneGroupOut()

    all_scores, all_perm_scores, all_pvals = [], [], []
    for freq_idx in range(X.shape[0]):
        scores, perm_scores, pvals = [], [], []
        for chan_idx in range(X.shape[-1]):
            X_sf = X[freq_idx,:,chan_idx]
            score, permutation_scores, pvalue = permutation_test_score(clf, X=X_sf.reshape(-1, 1), y=y, groups=groups, cv=cv, n_permutations=n_perms, n_jobs=-1)
            scores.append(score)
            perm_scores.append(permutation_scores)
            pvals.append(pvalue)
            print(f'Computed feature {freq_idx} chan {chan_idx} Score {score}, pvalue {pvalue}')
        all_scores.append(scores)
        all_perm_scores.append(perm_scores)
        all_pvals.append(pvals)
        
    all_scores = np.array(all_scores)
    all_perm_scores = np.array(all_perm_scores)
    all_pvals = np.array(all_pvals)
    all_results = {'scores': all_scores, 'perm_scores': all_perm_scores, 'pvals': all_pvals}
    return all_results